In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import sys
sys.path.append('/Users/david/miniconda3/lib/python3.8/site-packages')

import cv2 as cv

from ipywidgets import interact
import ipywidgets as widgets

%matplotlib auto

Using matplotlib backend: MacOSX


In [2]:
def filtre(w, h, dim):
    
    xpx, ypx = dim[0], dim[1]
    almostSquare = w > 0 and .9 < h/w < 1.1
    relevantDim = (.05*xpx < w < .7*xpx) and (.05*ypx < h < .7*ypx)
    
    return almostSquare and relevantDim

In [3]:
def reframe(img):
    ret, image = cv.threshold(img, 100, 255, 0)
    dim = image.shape
    cimg = cv.cvtColor(image, cv.COLOR_GRAY2RGB)
    contours, h = cv.findContours(image, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
    
    X, Y, W, H = 0, 0, 0, 0
    
    for cnt in contours:
        x, y, w, h = cv.boundingRect(cnt)       
        if filtre(w, h, dim) and w*h > W*H:
            X, Y, W, H = x, y, w, h
            
    xmin = max(0, X - 40)
    ymin = max(0, Y - 40)
    xmax = min(dim[1], X + W + 41)
    ymax = min(dim[0], Y + H + 41)
    
    return ymin, ymax, xmin, xmax

In [4]:
def drawEllipse(image, ymin, ymax, xmin, xmax, show=True):
    dim = image.shape
    cimg = cv.cvtColor(image, cv.COLOR_GRAY2RGB)
    contours, h = cv.findContours(image, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
    
    elli = []
    
    total = 0
    good = 0
    
    for cnt in contours:
        
        if cnt.shape[0] > 4:
            el = cv.fitEllipse(cnt)
            (cx, cy), (a, b), alpha = el
            if filtre(a, b, dim):
                cv.ellipse(cimg, el, (255, 0, 0), 5)
                good += 1
                if b > a:
                    a, b = b, a
                    alpha = (alpha + 90)%180
                elli.append([cx, cy, a, b, alpha])    

            total += 1

    if show:
        plt.imshow(cimg[ymin:ymax, xmin:xmax, :], cmap='Greys', aspect='equal')
        plt.axis('off')
    
    return good, total, np.array(elli)

In [5]:
part_path = 'retro.bmp'
path = 'img/Reelles/' + part_path
img = cv.imread(path, 0)
#img = cv.bilateralFilter(img, 16, 75, 75)
ymin, ymax, xmin, xmax = reframe(img)

plt.figure()

def Ecallback(v: int):
    ret, thresh = cv.threshold(img, v, 255, 0)
    #if img[:30, :].mean() > 127: # partie du fond
        #thresh = 255 - thresh
    drawEllipse(thresh, ymin, ymax, xmin, xmax, True)
    
interact(Ecallback, v=widgets.IntSlider(min=0, max=255, step=1, value=100))

interactive(children=(IntSlider(value=100, description='v', max=255), Output()), _dom_classes=('widget-interac…

<function __main__.Ecallback(v: int)>

# Variables d'intérêt

In [6]:
n = 81
x = np.arange(60, 141)

In [7]:
def addPlot(axes, xv, y1, y2, label1, label2, title):
    axes.scatter(xv, y1, label=label1, s=5, marker='^')
    axes.scatter(xv, y2, label=label2, s=5, marker='v')
    axes.set_title(title)
    axes.legend()

In [8]:
def Eadd():
    
    goods = np.zeros(n)
    totals = np.zeros(n)
    smallA = np.zeros(n)
    smallB = np.zeros(n)
    bigA = np.zeros(n)
    bigB = np.zeros(n)
    smallCX = np.zeros(n)
    smallCY = np.zeros(n)
    bigCX = np.zeros(n)
    bigCY = np.zeros(n)
    smallAngle = np.zeros(n)
    bigAngle = np.zeros(n)
    
    # Calculs

    for i, v in enumerate(x):
        
        ret, thresh = cv.threshold(img, v, 255, 0)
        if thresh.mean() < 127:
            thresh = 255 - thresh
        g, t, e = drawEllipse(thresh, ymin, ymax, xmin, xmax, False)
        goods[i] = g
        totals[i] = t

        if g == 2:
            
            a = e[:, 2]
            b = e[:, 3]
            I = np.argmax(a)
            
            smallA[i] = a[1 - I]
            bigA[i] = a[I]
            smallB[i] = b[1 - I]
            bigB[i] = b[I]
            
            c = e[:, :2]
            smallCX[i] = e[1 - I, 0]
            smallCY[i] = e[1 - I, 1]
            bigCX[i] = e[I, 0]
            bigCY[i] = e[I, 1]
            smallAngle[i] = e[1 - I, 4]
            bigAngle[i] = e[I, 4]
            
    # Return
    
    m = goods == 2
    T = x[m]
    concentric = np.sqrt(np.power(bigCX - smallCX, 2) + np.power(bigCY - smallCY, 2))
    concentric = concentric[m]
    smallEx = np.sqrt(1. - np.power(smallB[m]/smallA[m], 2))
    bigEx = np.sqrt(1. - np.power(bigB[m]/bigA[m], 2))         
            
    # Représentation
    
    fig_inf, ax_inf = plt.subplots(2)
    
    ax_inf[0].plot(x, goods)
    ax_inf[0].set_title('Ellipses pertinentes')
    
    ax_inf[1].plot(x, totals)
    ax_inf[1].set_title('Ellipses au total')
    
    for ax in ax_inf.flat:
        ax.label_outer()
    
    fig_inf.tight_layout();
    
    # Répresentation des mesures
    
    fig, ax = plt.subplots(2, 3, figsize=(14, 7))
    addPlot(ax[0, 0], T, smallA[m], smallB[m], 'Grand axe', 'Petit axe', 'Petite ellipse')
    addPlot(ax[1, 0], T, bigA[m], bigB[m], 'Grand axe', 'Petit axe', 'Grande ellipse')
    ax[0, 1].scatter(T, smallCX[m], label='$x_{\Omega}$(petite ellipse)', s=5, marker='^')
    ax[0, 1].scatter(T, smallCY[m], label='$y_{\Omega}$(petite ellipse)', s=5, marker='v')
    ax[0, 1].scatter(T, bigCX[m], label='$x_{\Omega}$(grande ellipse)', s=5, marker='x', color='tab:gray')
    ax[0, 1].scatter(T, bigCY[m], label='$y_{\Omega}$(grande ellipse)', s=5, marker='+', color='tab:red')
    ax[0, 1].set_title('Propriété des centres')
    ax[0, 1].legend()
    ax[1, 1].scatter(T, concentric, s=5, marker='x')
    ax[1, 1].set_title('Écart de concentricité')
    addPlot(ax[0, 2], T, smallAngle[m], bigAngle[m], 'Petite ellipses', 'Grande ellipse', 'Angles')
    addPlot(ax[1, 2], T, smallEx, bigEx, 'Petite ellipse', 'Grande ellipse', 'Excentricités')
    
    fig.tight_layout();
            
    return T, concentric, smallEx, bigEx

In [9]:
T, c, se, be = Eadd()